## Train File Using VGG-16 Model for Object Detection

### Imports 

In [1]:
import tensorflow as tf
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
from sklearn.model_selection import train_test_split
import cv2
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Activation, MaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import math
import numpy.random as npr
import os

### Code for checking for GPU built with CUDA

In [2]:
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)

Num GPUs Available 1
2.10.1


In [3]:
import sys
sys.version

'3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]'

In [ ]:
def train(train_data,labels,learning_rate = 0.01, steps_per_epoch =10,epochs = 100, patience = 10):
    
    ###########################################################################################################################
    #Cretes directory and file structure for CNN training
    ###########################################################################################################################
    '''A directory structure is created where files euqal to the number of classes are created inside the main directory.
       This file structure is necessary to generate trainable dataset using ImageDatagenerator. Where the name of each file 
       in the directory is the name of each classes in the dataset.
    '''
    # gets path of current directory
    cwd = os.getcwd()                                           
    if os.path.exists(cwd+'/sosimple_data') == False :
        # create a directory as given path and of given name
        os.mkdir(cwd+'/sosimple_data')                          

    for i in range(10):
        if os.path.exists(cwd+'/sosimple_data/'+ str(i)) == False: 
            # create a directory as given path and of given nam
            os.mkdir(cwd+'/sosimple_data/'+ str(i))
            
    ###########################################################################################################################
    # Convert to image and save in directory sosimple_data
    ###########################################################################################################################
    '''The data from .npy file of the image data is converted into .png image format and then saved into the respective directory
       according to the classes they have. Each image of same class is saved into same file into the main class.'''
    
    # Loads numpy file from the given path 
    data_train_t = np.load(train_data)                                   
    labels_train_t = np.load(labels)
    
    labels_names = ['x','square root','plus sign','negative sign',
                'equal','percent','partial','product','pi','summation']
    
    
    def class_convert_to_image(label, X, t, class_names, figsize=(15,150)): 

        # Index locations where class label stored in data_train
        sample_idx = np.where(t==label)[0] 

        #Number of samples in class label
        N = len(sample_idx)

        for i in range(N):
            imgname = str(label)+'_'+str(i)
            save_path = os.path.join(cwd+'/sosimple_data/',str(label),imgname+".png")
            plt.imsave(save_path,X[:,sample_idx[i]].reshape(300,300), cmap='gray')
        print("finished")
   
    # Converts array into images and saves them into there respective class files.
    for i in range(0,10):
        class_convert_to_image(i, data_train_t, labels_train_t, labels_names)
    
    ###########################################################################################################################
    # Pre-processing and input of image data from directory
    ###########################################################################################################################
    #gets the loaction of directory were imgae file are saved.
    train_dir = cwd +"/sosimple_data"
    
    # Used for data augmentation and preprocessing
    train_datagen = ImageDataGenerator(validation_split = 0.3) ## Converts the image files into a format that can be used to train 
                                                               ## CNN models also can be used for data augmentaion like rotate,shear
                                                               ## zoom,etc. Also splits data into train and validation split
    
    train_data = train_datagen.flow_from_directory(directory = train_dir,  ## Gets the data form the directory, used for image
                                                   target_size = (100,100),## preprocessing and divides the data into bacthes .
                                                   batch_size = 24,
                                                   subset='training')
    test_data = train_datagen.flow_from_directory(directory = train_dir, 
                                                  target_size = (100,100),
                                                  batch_size = 24,
                                                  subset='validation')
    ###########################################################################################################################
    # VGG -16 Model
    ###########################################################################################################################
    model = Sequential()
    # 1st Convolution Layer
    model.add(Conv2D(input_shape=(100,100,3), filters=64, kernel_size=(3,3), padding='same', activation='relu'))
    # 2nd Convolution layer
    model.add(Conv2D(filters=64, kernel_size=(3,3),padding='same', activation='relu'))
    # 3rd Pooling Layer
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    # 4th Convolution Layer
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
    # 5th Convolution layer
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
    # 6th Pooling layer
    model.add(MaxPooling2D(strides=(2,2), pool_size=(2,2)))
    # 7th Convolution layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    # 8th Convolution layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    # 9th Convolution layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    # 10th Pooling Layer
    model.add(MaxPooling2D(strides=(2,2), pool_size=(2,2)))
    # 11th Convolution layer
    model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='same', activation='relu'))
    # 12th Conv layer
    model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='same', activation='relu'))
    # 13th Conv layer
    model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='same', activation='relu'))
    # 14th Pooling Layer 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    # 15th Convolution layer
    model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='same', activation='relu'))
    # 16th Conv layer
    model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='same', activation='relu'))
    # 17th Conv layer
    model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='same', activation='relu'))
    # 18th Pooling Layer 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.summary()
    
    ############################################################################################################################
    # Model Checkpoints
    ############################################################################################################################
    mc = ModelCheckpoint(filepath="./train_model.h5", ## creates a checkpoint so that it can be used to train the data from were
                     monitor = "val_accuracy",         ## it stoped rather than just training it again from scratch also helps
                     verbose = 1,                      ## save weights of the model.
                     save_best_only = "True")
    es = EarlyStopping(monitor = "val_accuracy",           ## Used to stop training the model if there is no progress in given metric
                       min_delta = 0.01,                   ## helps avoid overfitting and better control of data
                       patience = patience,
                       verbose = 1)
    cb = [mc,es]

    
    ############################################################################################################################
    # Model Compilation
    ############################################################################################################################
    opt = SGD(learning_rate=learning_rate) ## Stocastic Gradient Descent this works better with VGG16 alos learning rate is hypeparameter
    model.load_weights('./train_model.h5') ## Loading the weights from the eposchs with best validation accuracy
    model.compile( optimizer=opt, loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])##compiling the model for training
    
    ##############################################################################################################################
    #Model training
    ###############################################################################################################################
    
    his = model.fit_generator(train_data,steps_per_epoch = steps_per_epoch , epochs= epochs , validation_data=test_data, callbacks = cb)
    
    ################################################################################################################################
    # Visualization of model performance
    ################################################################################################################################
    
    # loads the model and its weights along with all the epoches information in latest traiing
    model = load_model("./train_model.h5")
    h = his.history
    plt.plot(h['loss'], c = "red")
    plt.plot(h['val_loss'],'go--', c = "red")
    plt.plot(h['accuracy'], c = "blue")
    plt.plot(h['val_accuracy'],'go--', c = "blue")

    plt.title("Loss vs accuracy")
    plt.show()

In [ ]:
data_train_t = 'data_train_correct.npy'
labels_train_t = 'labels_train_corrected.npy'
train(data_train_t, labels_train_t,0.01,10,100,5)